In [1]:
from datetime import datetime
from elasticsearch import Elasticsearch
from elasticsearch.helpers import scan
from elasticsearch.helpers import bulk
es = Elasticsearch(['atlas-kibana.mwt2.org:9200'],timeout=60)

### find all the combinations of sources and destinations

# sorting times is messed up it needs a fix

In [8]:
ind = "network_weather-2017.1.11"

my_query = \
{
  "size": 0,
  "aggs": {      
        "type_counts": {
            "terms": { "field": "_type" },
            "aggs": {      
                "src_counts": {
                     "terms": { "field": "src" , "size": 10000},
                     "aggs": {      
                        "dest_counts": {
                        "terms": { "field": "dest" , "size": 10000}
                        }
                     }
                  }
              }
        }
  }
}
response = es.search(index=ind, body=my_query, request_timeout=1200)
#print(response['aggregations']['type_counts']['buckets'])

In [9]:
# for cleaning specific links
limitTo=['192.41.230.60','192.41.236.32','192.41.230.59','192.41.236.31','128.142.223.247','128.142.223.246']

In [13]:

types = response['aggregations']['type_counts']['buckets']

for t in types:
    allPairs=0
    print(t['key'])
    if not t['key']=='traceroute': continue
    for s in t['src_counts']['buckets']:
        bulk_body = []
        dcounter=0
        for d in s['dest_counts']['buckets']:
            allPairs += 1
            src=s['key']
            dest=d['key']
            #if src not in limitTo and dest not in limitTo: continue
            #print ('s: ',src,'d: ', dest, '\t docs:', d['doc_count'])
            myscroll={
                "size":100000,
            #    "_source": {
            #        "includes": [ "timestamp" ]
            #    },
#               "fields": ["timestamp", "delay_mean","throughput","packet_loss","src","dest"],
                "query":{
                    "bool" : {
                        "must" : [
                            {"term": {"_type": {"value": t['key'] } } },
                            {"term": {"src":   {"value": src } } },
                            {"term": {"dest":  {"value": dest } } }
                        ]
                    }
                }
            ,"sort": [{ "timestamp": { "order": "asc", "unmapped_type": "boolean" }}]
            }
            
            toDelete={}
#             oldres=0
            oldtime=""
            counter=0
            scroll = scan(es, query=myscroll, index=ind, size=5000, scroll='240s', request_timeout=120)
            for res in scroll:
                if counter<10: print(res)
                if res['_source']['timestamp']==oldtime:# and res['fields']['src']==src and res['fields']['dest'][0]==dest and res['fields']['packet_loss'][0]==oldres:
                    toDelete[res['_id']]=res['_index']
#                     print(res['fields'])
#                 oldres=res['fields']['packet_loss']
                oldtime=res['_source']['timestamp']
                counter+=1
#                if counter>200:
#                    break
#                 if counter%100==0: 
            break
            #print ("records:", counter, '\tto delete:',len(toDelete))
            for x in toDelete:
                if dcounter>100000:
                    dcounter=0
                    es.bulk('\n'.join(bulk_body), request_timeout=1200)
                    bulk_body=[]
                    print('deleted 100k')
                dcounter+=1
                bulk_body.append('{"delete": {"_id": "'+x+'", "_index":"'+toDelete[x]+'", "_type":"'+t['key']+'"}}')
#             print (bulk_body)
        if len(toDelete):
            print("final delete", dcounter)
            es.bulk('\n'.join(bulk_body), request_timeout=1200)

    print(t['key'],'Pairs:',allPairs)
print ('DONE.')

packet_loss_rate
latency
traceroute
{'_id': 'AVmLX0xt1yReIzHCm6vW', 'sort': [2951], '_index': 'network_weather-2017.1.11', '_score': None, '_type': 'traceroute', '_source': {'MA': '200.17.30.135', 'rtts': [0.0, 0.835, 5.218, 183.959, 183.893, 242.274, 241.963, 203.406, 203.024, 245.811, 217.305, 217.273, 216.297], 'dest': '109.105.125.232', 'hops': [None, '143.107.0.245', '143.107.151.114', '62.40.126.173', '62.40.126.234', '109.105.97.126', '109.105.97.132', '109.105.97.57', '109.105.102.117', '109.105.124.50', '109.105.124.51', '109.105.124.51', '109.105.125.232'], 'timestamp': '20170111T031428+0000', 'srcProduction': False, 'destVO': '', 'ttls': [1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12, 13], 'src': '200.17.30.135', 'destSite': 'FI_HIP_T2', 'hash': 5983927945936759295, 'destProduction': False}}
{'_id': 'AVmLX0xt1yReIzHCm6vX', 'sort': [2952], '_index': 'network_weather-2017.1.11', '_score': None, '_type': 'traceroute', '_source': {'MA': '200.17.30.135', 'rtts': [0.0, 0.795, 0.845, 183.8

ValueError: Empty value passed for a required argument 'body'.